In [1]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque

import numpy as np

import math
import datetime as dt
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
import pandas as pd
import cufflinks as cf

import Quandl as quandl

import plotly.plotly as py
import plotly.graph_objs as go
%matplotlib inline

import copy

In [322]:
ticker = 'BA'

stock = 'WIKI/' + ticker

stockseries2 = quandl.get(stock, authtoken="1Cx13bkj4vDb7E13GLD6")['Adj. Close'].ix['01/01/2014':'04/30/2016'].dropna()

In [14]:
X = [[1,2,3],[4,5,6],[7,8,9]]
y = [0,1,0]

rfmodel = RandomForestClassifier().fit(X, y)

rfmodel.predict(np.array([1,3,5]).reshape(1,-1))

array([1])

In [328]:
w = 30
X = []
y = []

def rfcreate(ar):
    global X
    global y
    global w
    
    changes = np.diff(ar) > 0
    X.append(changes[:-1])
    y.append(changes[-1])
    
    rfmodel = RandomForestClassifier().fit(np.array(X), np.array(y))

    return int(rfmodel.predict(changes[1:].reshape(1,-1)))


predictions =  stockseries2.rolling(window=20, center=False).apply(rfcreate).dropna()





In [282]:
info = pd.concat([stockseries, (stockseries.pct_change() > 0), predictions.shift().apply(bool)], axis=1)

In [302]:
TruePos = ((stockseries.pct_change() > 0) & predictions.shift().apply(bool))[
    ((stockseries.pct_change() > 0) & predictions.shift().apply(bool))]

TruePos = TruePos.astype(int)-1

In [303]:
predictions = predictions[predictions == 1] - 1

In [304]:

trace0 = go.Scatter(
    x = stockseries.index,
    y = stockseries.pct_change().values,
    name = ticker + ' Percent Change'
)

trace1 = go.Scatter(
    x = predictions.index,
    y = predictions.values,
    mode = 'markers',
    name = 'Buy Signal'
)

trace2 = go.Scatter(
    x = TruePos.index,
    y = TruePos.values,
    mode = 'markers',
    name = 'True Positives'
)

data = [trace0, trace1, trace2]

layout = dict(
    title = 'Random Forests Prediction'
)

fig = dict(data=data, layout=layout)

py.iplot(fig, filename = 'Random Forest')


In [306]:
tickerlist = ['AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'KO', 
             'DD', 'XOM', 'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 
             'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG',
             'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'DIS']

stock = 'WIKI/AAPL'

stockseries = quandl.get(stock, authtoken="1Cx13bkj4vDb7E13GLD6")['Adj. Close'].ix['01/01/2014':'04/30/2016']
for t in tickerlist:
    stock = 'WIKI/' + t
    getstock = quandl.get(stock, authtoken="1Cx13bkj4vDb7E13GLD6")['Adj. Close'].ix['01/01/2014':'04/30/2016']
    stockseries = pd.concat([stockseries, getstock], axis=1)
        
        
tickerlist = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'KO', 
             'DD', 'XOM', 'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 
             'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG',
             'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'DIS']

stockseries = stockseries.dropna()

stockseries.columns = tickerlist
        
        
        
        

In [337]:
w = 50


stocklist = []
successlist = []


for col in stockseries.columns:
    
    w = 30
    X = []
    y = []
    
    usedstock = stockseries[col]
    
    predictions =  usedstock.rolling(window=w, center=False).apply(rfcreate).dropna()
    
    TruePos = ((usedstock.pct_change() > 0) & predictions.shift().apply(bool))[
    ((usedstock.pct_change() > 0) & predictions.shift().apply(bool))]
    
    stocklist.append(col)
    successlist.append(len(TruePos)/len(predictions))
        
successtable = pd.DataFrame(dict(stock = stocklist, accuracy = successlist))


In [342]:
successtable.sort_values('accuracy', ascending=False)

,accuracy,stock
29,0.291045,DIS
17,0.283582,MMM
6,0.277985,KO
24,0.276119,UNH
10,0.264925,GS
20,0.261194,NKE
15,0.261194,JPM
7,0.255597,DD
23,0.253731,TRV
2,0.251866,BA
